# Deploying agent to reasoning engine

This notebook covers how we can restructure the agent code and ultimately deploy the agent to google vertex ai.

In [1]:
import requests
from typing import Optional, List
from IPython.display import display, Markdown

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langchain.memory import ChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

from langchain_google_cloud_sql_pg import (
    PostgresEngine,
    PostgresVectorStore,
)

from vertexai.preview import reasoning_engines 
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

In [2]:
project_id = "imrenagi-gemini-experiment"  # @param {type:"string"}
region = "us-central1" #change this to project location
staging_bucket_name = "courses-imrenagicom-agent" #change this with your staging bucket name
instance_name="pyconapac-demo"
database_password = 'testing'
database_name = 'testing'
database_user = 'testing'

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

# dont update variable below

cloudrun_services = !gcloud run services describe courses-api --region=us-central1 --format='value(status.url)'
api_base_url = cloudrun_services[0]

staging_bucket_uri = f"gs://{staging_bucket_name}"
# get the ip address of the cloudsql instance
ip_addresses = !gcloud sql instances describe {instance_name} --format="value(ipAddresses[0].ipAddress)"
database_host = ip_addresses[0]

gemini_embedding_model = "text-embedding-004"
gemini_llm_model = "gemini-1.5-pro"
embeddings_table_name = "course_content_embeddings"

print(f"API Base URL: {api_base_url}")
print(f"Database Host: {database_host}")

API Base URL: https://courses-api-uzttxm4diq-uc.a.run.app
Database Host: 34.133.247.116


In [3]:
import vertexai
vertexai.init(project=project_id, location=region, staging_bucket=staging_bucket_uri)

### Agent Code

This section below define a few classes that we have tried from the previous notebook.

The main focus here should be `CourseAgent` where we setup all dependencies required by the agent

In [4]:
class CourseAPIClient:
  def __init__(self, url=api_base_url):
    self.url = url
    
  def list_courses(self):
      response = requests.get(f"{self.url}/courses")
      return response.json()

  def get_course(self, course_name):
      response = requests.get(f"{self.url}/courses/{course_name}")
      return response.json()

  def create_order(self, course, user_name, user_email):
      payload = {
          "course": course,
          "user_name": user_name,
          "user_email": user_email
      }
      response = requests.post(f"{self.url}/orders", json=payload)
      return response.json()

  def get_order(self, order_id):
      response = requests.get(f"{self.url}/orders/{order_id}")
      return response.json()

  def pay_order(self, order_id):
      response = requests.post(f"{self.url}/orders/{order_id}:pay")
      return response.json()

  def get_payment_page_url(self, order_id):
      return f"{self.url}/orders/{order_id}/payment"

class GetCourseInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")

class GetOrderInput(BaseModel):
    order_number: str = Field(description="order number identifier. this is a unique identifier in uuid format.")

class CreateOrderInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")
    user_name: str = Field(description="name of the user who is purchasing the course .")
    user_email: str = Field(description="email of the user who is purchasing the course.")

class CourseAgent(reasoning_engines.Queryable):
    def __init__(
        self,
        model: str,
        project: str,
        region: str,
        instance: str,
        database: str,
        table: str,
        user: Optional[str] = None,
        password: Optional[str] = None,
        
    ):
        self.model_name = model
        self.project = project
        self.region = region
        self.instance = instance
        self.database = database
        self.table = table
        self.user = user
        self.password = password                
        self.store = {}   
        self.agent = None
        self.retriever = None     

    def __getstate__(self):
        """Custom method for pickling the object."""
        state = self.__dict__.copy()
        # Remove the unpicklable entries
        del state['agent']
        del state['retriever']
        return state

    def __setstate__(self, state):
        """Custom method for unpickling the object."""
        self.__dict__.update(state)
        self.agent = None
        self.retriever = None
        # Note: set_up() will need to be called after unpickling


    def list_courses(self) -> List[str]:
        """List all available courses sold on the platform."""        
        client = CourseAPIClient()
        return client.list_courses()

    def create_order(self, course: str, user_name: str, user_email: str) -> str:
        """Create order for a course. This function can be used to create an order for a course. When this function returns successfully, it will return payment url to user to make payment. """        
        client = CourseAPIClient()
        print(f"Creating order for course: {course}, user_name: {user_name}, user_email: {user_email}")
        
        res = client.create_order(course, user_name, user_email)
        print(res)
        order_id = res["order_id"]
        payment_url = f"{api_base_url}/orders/{order_id}/payment"
        return f"Order number {order_id} created successfully. Payment URL: {payment_url}"

    def get_course(self, course: str) -> str:        
        """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc."""        
        client = CourseAPIClient()
        return client.get_course(course)
    
    def get_order(self, order_number: str) -> str:
        """Get order by using order number. This function can be used to get order details such as payment status to check whether the order has been paid or not. If user already paid the course, say thanks"""        
        client = CourseAPIClient()
        return client.get_order(order_number)

    def search_course_content(self, query: str) -> str:
        """Explain about software security course materials."""
        result = str(self.retriever.invoke(query))
        return result        

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
            if session_id not in self.store:
                self.store[session_id] = ChatMessageHistory()
            return self.store[session_id]        

    def set_up(self):
        """All unpickle-able logic should go here.
        In general, add any logic that requires a network or database
        connection.
        """

        # Initialize the vector store
        engine = PostgresEngine.from_instance(
            self.project,
            self.region,
            self.instance,
            self.database,
            user=self.user,
            password=self.password,
            quota_project=self.project,
        )

        embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)
        
        vector_store = PostgresVectorStore.create_sync(
            engine,
            table_name=self.table,
            embedding_service=embeddings_service,
        )
        self.retriever = vector_store.as_retriever(search_kwargs={"k": 10})
                
        search_course_content = StructuredTool.from_function(
            func=self.search_course_content,
            name="search_course_content",
            description="Explain about software security course materials.",            
        )

        list_courses = StructuredTool.from_function(
            func=self.list_courses,
            name="list_courses",
            description="List all available courses sold on the platform.",
        )

        get_course = StructuredTool.from_function(
            func=self.get_course,
            name="get-course-tool",
            description="""Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
        This function can be used to get course details such as course price, etc.""",
            args_schema=GetCourseInput,
        )

        create_order = StructuredTool.from_function(
            func=self.create_order,
            name="create-order-tool",
            description="""Create order for a course. This function can be used to create an order for a course. When this function returns successfully, it will return payment url to user to make payment.""",
            args_schema=CreateOrderInput,
        )

        get_order = StructuredTool.from_function(
            func=self.get_order,
            name="get-order-tool",
            description="""Get order by using order number. This function can be used to get order details such as payment status to check whether the order has been paid or not. If user already paid the course, say thanks""",
            args_schema=GetOrderInput,
        )
        
        tools = [search_course_content, list_courses, get_course, create_order, get_order]

        # Initialize the LLM and prompt
        prompt = {
            "chat_history": lambda x: x["history"],
            "input": lambda x: x["input"],
            "agent_scratchpad": (
                lambda x: format_to_openai_function_messages(x["intermediate_steps"])
            ),
        } | ChatPromptTemplate(
            messages = [
                SystemMessagePromptTemplate.from_template("""
                You are a bot assistant that sells online course about software security. You only use information provided from datastore or tools. You can provide the information that is relevant to the user's question or the summary of the content. If they ask about the content, you can give them more detail about the content. If the user seems interested, you may suggest the user to enroll in the course. 
                """),
                MessagesPlaceholder(variable_name="chat_history", optional=True),
                HumanMessagePromptTemplate.from_template("Use tools to answer this questions: {input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        safety_settings = {
            HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        }

        ## Model parameters
        model_kwargs = {
            "temperature": 0.5,
            "safety_settings": safety_settings,
        }


        self.agent = reasoning_engines.LangchainAgent(
            model=self.model_name,
            tools=tools,
            prompt=prompt,    
            chat_history=self.get_session_history,
            agent_executor_kwargs={
                "return_intermediate_steps": True,
            },
            model_kwargs=model_kwargs,
            enable_tracing=False,
        )
        print("agent is configured")


    def query(self, input: str, session_id: str) -> str:
        """Query the application.

        Args:
            input: The user query.
            session_id: The user's session id.

        Returns:
            The LLM response dictionary.
        """
        response = self.agent.query(
            input=input,
            config={"configurable": {"session_id": session_id}},
        )
        return response

### Instantiating the agent

In [5]:
agent = CourseAgent(
    model=gemini_llm_model,
    project=project_id,
    region=region,
    instance=instance_name,
    database=database_name,
    table=embeddings_table_name,    
    user=database_user,
    password=database_password,           
)
agent.set_up()

agent is configured


### Testing agent (again) locally

In [6]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

Generated session ID: 3f4efadc-f1af-4af6-9524-3eabf57a11b4


In [7]:
res = agent.query(
    input="Can you please share what are taught on this course?", 
    session_id=session_id)

display(Markdown(res["output"]))

This course provides cheat sheets on various topics in software security, including:

* **Authorization:** Learn best practices for implementing authorization logic, covering topics like role-based access control (RBAC), attribute-based access control (ABAC), and more.
* **REST Security:** Understand how to secure RESTful APIs, including handling state, using HTTP verbs and error codes securely, and protecting against common attacks.
* **File Upload Security:** Discover how to securely handle file uploads, including validating file content, preventing malicious uploads, and protecting against vulnerabilities like directory traversal. 

This is just a glimpse of what you'll learn.  Would you like to know more? 


### Deploying the Agent on Vertex AI

Deploying is as simple as calling `create()` method. We will provide the agent here and some dependencies required to run the agent.

In [8]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform==1.69.0",
        "google-cloud-aiplatform[langchain]",
        "google-cloud-aiplatform[reasoningengine]",
        "langchain==0.2.16",
        "langchain_core==0.2.39",
        "langchain_community==0.2.17",
        "langchain-google-vertexai==1.0.10",
        "cloudpickle==3.1.0",
        "pydantic==2.9.2",
        "langchain-google-community==1.0.8",
        "google-cloud-discoveryengine==0.12.3",
        "nest-asyncio",
        "asyncio==3.4.3",
        "asyncpg==0.29.0",
        "cloud-sql-python-connector[asyncpg]==1.12.1",        
        "langchain-google-cloud-sql-pg==0.11.0",
        "numpy",
        "pandas",
        "pgvector==0.3.5",
        "psycopg2-binary==2.9.9",
        "requests",
        "google-cloud-trace"
    ],    
    display_name="course-agent",
    sys_version="3.11",
)
remote_agent

Using bucket courses-imrenagicom-agent
Writing to gs://courses-imrenagicom-agent/reasoning_engine/reasoning_engine.pkl
Writing to gs://courses-imrenagicom-agent/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://courses-imrenagicom-agent/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/896489987664/locations/us-central1/reasoningEngines/5811983280051847168/operations/3862336580303716352
ReasoningEngine created. Resource name: projects/896489987664/locations/us-central1/reasoningEngines/5811983280051847168
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/896489987664/locations/us-central1/reasoningEngines/5811983280051847168')


resource name: projects/896489987664/locations/us-central1/reasoningEngines/5811983280051847168

In [9]:
project_id

'imrenagi-gemini-experiment'

In [10]:
# Retrieve the project number associated with your project ID
from googleapiclient import discovery
service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=project_id)
response = request.execute()
project_number = response["projectNumber"]
project_number

'896489987664'

In [11]:
!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.editor

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/discoveryengine.editor"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/cloudsql.admin"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/run.invoker"

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/editor"    

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role="roles/aiplatform.extensionServiceAgent"        

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-vertex-ex.iam.gserviceaccount.com \
    --role="roles/aiplatform.extensionServiceAgent"    

!gcloud projects add-iam-policy-binding {project_id} \
    --member=serviceAccount:service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com \
    --role="roles/aiplatform.ragServiceAgent"    

Updated IAM policy for project [imrenagi-gemini-experiment].
bindings:
- members:
  - user:imre.nagi2812@gmail.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-896489987664@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-896489987664@gcp-sa-vertex-ex-cc.iam.gserviceaccount.com
  role: roles/aiplatform.extensionCustomCodeServiceAgent
- members:
  - serviceAccount:service-896489987664@gcp-sa-vertex-ex.iam.gserviceaccount.com
  role: roles/aiplatform.extensionServiceAgent
- members:
  - serviceAccount:service-896489987664@gcp-sa-vertex-rag.iam.gserviceaccount.com
  role: roles/aiplatform.ragServiceAgent
- members:
  - serviceAccount:service-896489987664@gcp-sa-aiplatform-re.iam.gserviceaccount.com
  role: roles/aiplatform.reasoningEngineServiceAgent
- members:
  - serviceAccount:service-896489987664@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- mem

### Testing Remote Agent

In [12]:
import uuid

# Generate a UUID for the session ID
session_id = str(uuid.uuid4())
print(f"Generated session ID: {session_id}")

Generated session ID: a9349f5c-3037-40fd-905e-fee40ab1ec3d


In [13]:
# Testing the remote agent
response = remote_agent.query(
  input="Can you please share what are being taught on this course",
  session_id=session_id,
)
display(Markdown(response["output"]))

FailedPrecondition: 400 Reasoning Engine Execution failed.
Please navigate to the Cloud Console Log Explorer page (https://console.cloud.google.com/logs/query;query=resource.type="aiplatform.googleapis.com%2FReasoningEngine"%0Aresource.labels.reasoning_engine_id=~"5811983280051847168"?project=896489987664) to view the specific errors. Additionally, please check our troubleshooting guide (https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/troubleshooting/use) for more information, or visit our Cloud community forum (https://www.googlecloudcommunity.com/gc/AI-ML/bd-p/cloud-ai-ml) or GitHub repository (https://github.com/googleapis/python-aiplatform/issues) to find answers and ask for help.
Error Details: {"detail":"Traceback (most recent call last):\n  File \"/usr/local/lib/python3.11/site-packages/google/api_core/grpc_helpers.py\", line 170, in error_remapped_callable\n    return _StreamingResponseIterator(\n           ^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File \"/usr/local

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Does it teach about how to design a forgot password system securely?",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="How much this course costs?",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Yes. I want to enroll",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Name is Mulyono and email is mulyono@gmail.com",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Yes",
  session_id=session_id,
)
display(Markdown(response["output"]))

In [ ]:
# Testing the remote agent
response = remote_agent.query(
  input="Can you please check the status of my order",
  session_id=session_id,
)
display(Markdown(response["output"]))